In [1]:
import os
import sys
import urllib, io
os.getcwd()
sys.path.append("..")
sys.path.append("../utils")
sys.path.append("../analysis/utils")


import numpy as np
import scipy.stats as stats
import pandas as pd
from sklearn.cluster import AffinityPropagation
from sklearn.manifold import TSNE

import pymongo as pm
from collections import Counter
import json
import re
import ast

from PIL import Image, ImageOps, ImageDraw, ImageFont 

from io import BytesIO
import base64

import  matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

import seaborn as sns
sns.set_context('talk')
sns.set_style('darkgrid')

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

# import drawing_utils as drawing
import importlib
import scoring

In [2]:
## directory & file hierarchy
proj_dir = os.path.abspath('..')
datavol_dir = os.path.join(proj_dir,'data')
analysis_dir =  os.path.abspath('../')
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
json_dir = os.path.join(results_dir,'json')
exp_dir = os.path.abspath(os.path.join(proj_dir,'behavioral_experiments'))
png_dir = os.path.abspath(os.path.join(datavol_dir,'png'))

## add helpers to python path
if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       

In [3]:
# set vars 
auth = pd.read_csv(os.path.join(analysis_dir,'auth.txt'), header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'cogtoolslab.org'

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['compositional_abstractions']
coll = db['referring_expressions']

# which iteration name should we use?
iterationName = 'ca_ref_exp_prolific_block_tower_dev'

## look up number of trials (including paired-practice)
# numTrials = 13

In [4]:
query = coll.find({"$and":[
                        {'trial_type': 'survey-text'},
                        {'iterationName': iterationName}
                        ]})
df_survey = pd.DataFrame(query)
list(df_survey['response'])

KeyError: 'response'

In [5]:
query = coll.find({"$and":[
                        {'trial_type': 'ca-label-ref-exp'},
                        {'iterationName': iterationName}
                        ]})
df_trial = pd.DataFrame(query)

In [6]:
df_trial['responses'][0]

[{'msgNum': 0,
  'message': 'horizontal block middle of red line left side\n',
  'message_id': '852d2ee5-0061-fd30-d907-3286329cd43a',
  'counts': {'block': '1', 'tower': '0'},
  'refExps': 'horizontal block, '},
 {'msgNum': 1,
  'message': 'vertical block on top of this one on the left side',
  'message_id': '6a4ace0e-d89a-9500-9f13-d2fbe583847f',
  'counts': {'block': '1', 'tower': '0'},
  'refExps': 'vertical block, '},
 {'msgNum': 2,
  'message': 'one more the same on top of it',
  'message_id': 'ed0d0418-2d58-bce9-8974-5cd7111868f8',
  'counts': {'block': '1', 'tower': '0'},
  'refExps': 'one more the same, '},
 {'msgNum': 3,
  'message': 'now a horizontal one above that, it should go to the right',
  'message_id': '6a594b07-eda6-7806-3a19-607d3bfb255e',
  'counts': {'block': '1', 'tower': '0'},
  'refExps': 'horizontal one, '},
 {'msgNum': 4,
  'message': 'right red line, go left 2 blocks and there is a vertical block',
  'message_id': '00183cfe-ceb0-9220-7984-f33f61c61ae4',
  'c

## Wrangle dataframe into message by message

In [113]:
# df_trial['responses'] = df_trial['responses'].apply(ast.literal_eval)

In [114]:
def counts_to_series(df):
    return pd.DataFrame.from_records(df.counts)


def extract_counts(row):
    '''
    Extracts message responses from 
    '''
    df = pd.DataFrame.from_dict(row.responses)
    df.loc[:,'_id'] = row['_id']
    df = pd.concat([df, counts_to_series(df)], axis = 1)

    return df
    

In [115]:
df_ref_exp = pd.concat(list(df_trial.apply(extract_counts, axis=1))).reset_index(drop=True)
df_ref_exp_full = df_ref_exp.merge(df_trial, how='left', on='_id')

In [116]:
df_ref_exp

,msgNum,message,message_id,counts,_id,block,tower
0,0,put a blue block three from the left.,e3e70682-c209-4cac-629f-6fbed82c07cd,"{'block': '2', 'tower': '2'}",641a15598190ec15d752aeae,2,2
1,1,place a red block on top of the left side of the blue block and then another red on top of that red,f728b4fa-4248-5e3a-0a5d-2f346baa9455,"{'block': '2', 'tower': '1'}",641a15598190ec15d752aeae,2,1
2,2,Now place a blue block on top of that that lines up with the bottom blue block,eb1167b3-67a9-c378-7c65-c1e582e2e662,"{'block': '2', 'tower': '2'}",641a15598190ec15d752aeae,2,2
3,3,"remove that last blue block, place it on top of the red blocks",f7c1bd87-4da5-e709-d471-3d60c8a70639,"{'block': '1', 'tower': '2'}",641a15598190ec15d752aeae,1,2
4,4,place a red block on the second grid from the right of the screen\n,e443df78-9558-867f-5ba9-1faf7a024204,"{'block': '2', 'tower': '1'}",641a15598190ec15d752aeae,2,1
...,...,...,...,...,...,...,...
463,1,"Big L : Blue 8,1",f3f05323-3bd5-f8a2-073c-36ba5d393bf9,"{'block': '1', 'tower': '1'}",641a22388190ec15d752af4e,1,1
464,0,"Tall C : Blue 3, 1",88da091b-61f6-c877-0212-585b0b2f8580,"{'block': '1', 'tower': '1'}",641a223f8190ec15d752af4f,1,1
465,1,"Big L : blue 8, 1",d24e3104-1dd2-6234-2131-24bd01275a63,"{'block': '1', 'tower': '1'}",641a223f8190ec15d752af4f,1,1
466,0,"House : Red 2, 1",e998952c-ef58-951f-9b1a-e0199aca0c4a,"{'block': '1', 'tower': '4'}",641a22458190ec15d752af50,1,4


In [117]:
pd.set_option('display.max_colwidth', 0)

In [121]:
n += 30
df_ref_exp[['message','block','tower']][n:n+30]

,message,block,tower
90,place a blues right tile above the red going leftward,0,0
91,place another blue next to that blue,1,0
92,place another blue on same level as the first blue,1,0
93,L-shape with 2x red and 2x Blue starting in the second grid from the left,4,8
94,upsidedown U-shape starting in the second grid from the right,4,6
95,tall c-shape starting in third grid from left,2,4
96,upsidedown U-shape starting in the second grid from the right,2,6
97,place blue 2 to the right of left,0,2
98,place red above blues left most tile,1,0
99,place red above that red,1,0


In [45]:
df_chat = pd.read_csv('../results/csv/df_chat_cogsci21.csv')

In [46]:
df_chat

,Unnamed: 0,_id,iterationName,gameid,time,assignmentId,leftTarget,rightTarget,trialNum,turnNum,...,timeElapsedInTurn,timeElapsedInTrial,eventType,practice,individualPracticeAttempts,turnStartTime,trialStartTime,word_count,char_count,flagged
0,0,5ef78389ce2f7f58fcd0a350,pilot0,1494-029e2297-bd3f-4cfe-be00-d06d36724e4e,1.593279e+12,3E47SOBEYUW78Q4UOK6BV9E6YCECIM,vertical,horizontal,NaN,0,...,60567,60567,chatMessage,True,NaN,NaN,NaN,15,63,False
1,1,5ef783bace2f7f58fcd0a358,pilot0,1494-029e2297-bd3f-4cfe-be00-d06d36724e4e,1.593279e+12,3E47SOBEYUW78Q4UOK6BV9E6YCECIM,vertical,horizontal,NaN,0,...,20018,20018,chatMessage,True,NaN,NaN,NaN,13,70,False
2,2,5ef783e4ce2f7f58fcd0a35e,pilot0,1494-029e2297-bd3f-4cfe-be00-d06d36724e4e,1.593279e+12,3E47SOBEYUW78Q4UOK6BV9E6YCECIM,C,Pi,0.0,0,...,28166,28166,chatMessage,False,NaN,NaN,NaN,9,40,False
3,3,5ef78401ce2f7f58fcd0a365,pilot0,1494-029e2297-bd3f-4cfe-be00-d06d36724e4e,1.593279e+12,3E47SOBEYUW78Q4UOK6BV9E6YCECIM,C,Pi,0.0,2,...,21993,57512,chatMessage,False,NaN,NaN,NaN,23,99,False
4,4,5ef7842ace2f7f58fcd0a36e,pilot0,1494-029e2297-bd3f-4cfe-be00-d06d36724e4e,1.593280e+12,3E47SOBEYUW78Q4UOK6BV9E6YCECIM,C,Pi,0.0,4,...,24789,98193,chatMessage,False,NaN,NaN,NaN,17,78,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3114,31,5f590f181034572e66881404,pilot4b,9727-37a58c13-7178-412c-88d0-6b8841439106,1.599672e+12,3HL8HNGX491L01V1BSQZQM07RDF9FG,Pi,C,9.0,2,...,18109,61157,chatMessage,False,0.0,1.599672e+12,1.599672e+12,21,99,False
3115,32,5f590f3b1034572e6688140c,pilot4b,9727-37a58c13-7178-412c-88d0-6b8841439106,1.599672e+12,3HL8HNGX491L01V1BSQZQM07RDF9FG,Pi,L,10.0,0,...,16063,16063,chatMessage,False,0.0,1.599672e+12,1.599672e+12,15,78,False
3116,33,5f590f5e1034572e66881416,pilot4b,9727-37a58c13-7178-412c-88d0-6b8841439106,1.599672e+12,3HL8HNGX491L01V1BSQZQM07RDF9FG,Pi,L,10.0,2,...,19200,50916,chatMessage,False,0.0,1.599672e+12,1.599672e+12,16,77,False
3117,34,5f590fa51034572e66881422,pilot4b,9727-37a58c13-7178-412c-88d0-6b8841439106,1.599672e+12,3HL8HNGX491L01V1BSQZQM07RDF9FG,C,L,11.0,0,...,42453,42453,chatMessage,False,0.0,1.599672e+12,1.599672e+12,19,99,False


In [29]:
df_jj = pd.read_csv('../results/csv/JJ_content.csv')

In [30]:
df_jj

,gameid,time,assignmentId,leftTarget,rightTarget,trialNum,turnNum,repNum,message,timeElapsedInTurn,...,block_justin,toer_justin,scene_justin,Flagged,phrases_justin,block_julia,tower_juli,scene_juli,phrases_julia,content
0,1494-029e2297-bd3f-4cfe-be00-d06d36724e4e,1.593280e+12,3E47SOBEYUW78Q4UOK6BV9E6YCECIM,C,Pi,0,0,0,put a blue block three from the left.,28166,...,1,0,0,NaN,a blue block,1.0,0,0.0,a blue block,a blue block a blue block
1,1494-029e2297-bd3f-4cfe-be00-d06d36724e4e,1.593280e+12,3E47SOBEYUW78Q4UOK6BV9E6YCECIM,C,Pi,0,2,0,place a red block on top of the left side of t...,21993,...,2,0,0,NaN,"a red block, another red",2.0,0,0.0,"a red block, another red","a red block, another red a red block, another red"
2,1494-029e2297-bd3f-4cfe-be00-d06d36724e4e,1.593280e+12,3E47SOBEYUW78Q4UOK6BV9E6YCECIM,L,Pi,2,0,0,Do the same L shape on the left side as last t...,36762,...,4,1,0,NaN,"L shape, 2x blue, 2x red",4.0,1,0.0,"L shape, 2x blue, 2x red","L shape, 2x blue, 2x red L shape, 2x blue, 2x red"
3,1494-029e2297-bd3f-4cfe-be00-d06d36724e4e,1.593280e+12,3E47SOBEYUW78Q4UOK6BV9E6YCECIM,C,Pi,0,4,0,Now place a blue block on top of that that lin...,24789,...,1,0,0,NaN,a blue block,1.0,0,0.0,a blue block,a blue block a blue block
4,1494-029e2297-bd3f-4cfe-be00-d06d36724e4e,1.593280e+12,3E47SOBEYUW78Q4UOK6BV9E6YCECIM,C,Pi,0,6,0,"remove that last blue block, place it on top o...",29367,...,1,0,0,NaN,that last blue block,1.0,0,0.0,last blue block,that last blue block last blue block
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2138,5156-be8ae9d7-0f4f-4f69-9d5b-fa70394f9dea,1.599680e+12,3UN61F00H0PZ24YYZXDCVV9O86W5R7,C,Pi,7,0,2,"left- very tall skinny C, right - arch",13899,...,0,2,0,NaN,"very tall skinny C, arch",0.0,2,0.0,"very tall skinny C, arch","very tall skinny C, arch very tall skinny C, arch"
2139,5156-be8ae9d7-0f4f-4f69-9d5b-fa70394f9dea,1.599680e+12,3UN61F00H0PZ24YYZXDCVV9O86W5R7,L,C,8,0,2,"left- large L, right - very tall skinny C",19081,...,0,2,0,NaN,"large L, very tall skinny C",0.0,2,0.0,"large L, very tall skinny C","large L, very tall skinny C large L, very tall..."
2140,5156-be8ae9d7-0f4f-4f69-9d5b-fa70394f9dea,1.599680e+12,3UN61F00H0PZ24YYZXDCVV9O86W5R7,Pi,C,9,0,3,"left- arch, right - very tall skinny C",11690,...,0,2,0,NaN,"arch, very tall skinny C",0.0,2,0.0,"arch, very tall skinny C","arch, very tall skinny C arch, very tall skinny C"
2141,5156-be8ae9d7-0f4f-4f69-9d5b-fa70394f9dea,1.599680e+12,3UN61F00H0PZ24YYZXDCVV9O86W5R7,Pi,L,10,0,3,"you are rockin this! Left- arch, right- large L",21226,...,0,2,0,NaN,"arch, large L",0.0,2,0.0,"arch, large L","arch, large L arch, large L"
